In [41]:
import pandas as pd
import pybedtools
import os

In [40]:
# PARAMETERS
gwas="Table S2. Detailed summary statistics on all nominated risk variants, known and novel_.txt"
bed_dir="/anvil/projects/x-mcb130189/Wubin/BG/pseudobulk/DMR/Subclass/Neuron/bed"
plink_dir="/anvil/projects/x-mcb130189/rwang22/bican/ref/GRCh38/plink_files"

# filter to get the 90 significant loci 
These additional filters exclude variants that 1) had a random-effects P value across all datasets > 4.67E-04 and 2) a conditional analysis P > 4.67E-04 using participant level 23andMe genotype data.

In [3]:
all_variants = pd.read_csv(gwas, sep = "\t")
all_variants.head()

,SNP,CHR,BP,Nearest Gene,QTL Nominated Gene (nearest QTL),Effect allele,Other allele,Effect allele frequency,"Beta, all studies","SE, all studies",...,"Freq1, new studies","Beta, new studies","StdErr, new studies","P, new studies","I2, new studies",Passes pooled 23andMe QC,Known GWAS locus within 1MB,Failed final filtering and QC,Locus within 250KB,Locus Number
0,rs114138760,1,"154,898,185",PMVK,NaN,c,g,0.0112,0.2812,0.0478,...,0.0109,0.1997,0.0843,1.780000e-02,0.0,T,1,0,1.0,1.0
1,rs35749011,1,"155,135,036",KRTCAP2,NaN,a,g,0.0169,0.6068,0.0342,...,0.0177,0.6798,0.0615,2.160000e-28,0.0,T,1,0,1.0,1.0
2,rs76763715,1,"155,205,634",GBAP1,GBAP1,t,c,0.9953,-0.7467,0.0765,...,0.9941,-0.6693,0.1269,1.340000e-07,0.0,F,1,0,1.0,1.0
3,rs6658353,1,"161,469,054",FCGR2A,FCGR2A,c,g,0.5011,0.0650,0.0094,...,0.5105,0.0662,0.0171,1.100000e-04,46.4,T,0,0,0.0,2.0
4,rs11578699,1,"171,719,769",VAMP4,VAMP4,t,c,0.1949,-0.0704,0.0120,...,0.1947,-0.0663,0.0226,3.310000e-03,13.4,T,0,0,0.0,3.0


In [5]:
top_90 = all_variants[~all_variants["Locus Number"].isna()]

In [ ]:
# liftover to hg38
rows = []
for index, row in top_90.iterrows():
    chr = row["CHR"]
    bim_file = f'{plink_dir}/1000G.EUR.hg38.{chr}.bim'
    bim = pd.read_csv(bim_file, sep = "\t", header = None, 
                      names=["CHR", "SNP", "GD", "BP", "A1", "A2"])
    cur = bim[bim["SNP"] == row["SNP"]]
    if len(cur) == 0:
        print(f'CAN NOT FIND {row["SNP"]}')
    else:
        bp = int(row["BP"].replace(',', ''))
        rows.append({})

CAN NOT FIND rs76763715
CAN NOT FIND rs34637584


In [33]:
rows.append({'SNP' : "rs76763715", 'CHR': 1, 'BP': 155235843})
rows.append({'SNP' : "rs34637584", 'CHR': 12, 'BP': 40340400})
df = pd.DataFrame(rows)
df["START"] = df["BP"] - 1
df = df[['CHR', 'START', 'BP', 'SNP']]

In [34]:
df.head()

,CHR,START,BP,SNP
0,1,154898184,154898185,rs114138760
1,1,155135035,155135036,rs35749011
2,1,161469053,161469054,rs6658353
3,1,171719768,171719769,rs11578699
4,1,205723571,205723572,rs823118


# bed file intersect with hypo dmrs

In [53]:
hits = []
for bedfile in os.listdir(bed_dir):
    x = pd.read_csv(f'{bed_dir}/{bedfile}', header = None, sep = "\t", names = ["chr", "start", "end"])
    dmr = pybedtools.BedTool.from_dataframe(x)
    # print(dmr.head())
    snp = pybedtools.BedTool.from_dataframe(df)
    tmp = dmr.intersect(snp)
    if tmp.count() > 0:
        hits.append(bedfile)

***** WARNING: File /tmp/pybedtools.5q8osvh6.tmp has inconsistent naming convention for record:
chr1	53062	53562

***** WARNING: File /tmp/pybedtools.5q8osvh6.tmp has inconsistent naming convention for record:
chr1	53062	53562

***** WARNING: File /tmp/pybedtools.py14hsb4.tmp has inconsistent naming convention for record:
chr1	780748	781248

***** WARNING: File /tmp/pybedtools.py14hsb4.tmp has inconsistent naming convention for record:
chr1	780748	781248

***** WARNING: File /tmp/pybedtools.uaghexio.tmp has inconsistent naming convention for record:
chr1	19142	19642

***** WARNING: File /tmp/pybedtools.uaghexio.tmp has inconsistent naming convention for record:
chr1	19142	19642

***** WARNING: File /tmp/pybedtools.zq8b4bz7.tmp has inconsistent naming convention for record:
chr1	19142	19642

***** WARNING: File /tmp/pybedtools.zq8b4bz7.tmp has inconsistent naming convention for record:
chr1	19142	19642

***** WARNING: File /tmp/pybedtools.kjhnni4m.tmp has inconsistent naming convention 

In [54]:
hits

[]